In [0]:
import io
import json
import requests
import functools
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
pd.options.mode.chained_assignment = None

## Bitcoin price

In [0]:
def make_request(url, *args):
  print(url(*args))
  return requests.get(url(*args))

In [0]:
start      = 20100718
end        = 20180429
params     = "PriceUSD"
url        = lambda params: f"https://community-api.coinmetrics.io/v2/assets/btc/metricdata?metrics={params}&start={start}&end={end}"

In [0]:
response = make_request(url, "PriceUSD")

https://community-api.coinmetrics.io/v2/assets/btc/metricdata?metrics=PriceUSD&start=20100718&end=20180429


In [0]:
json = response.json()

In [0]:
price_df = pd.DataFrame(json["metricData"]["series"])
price_df["values"] = price_df["values"].apply(lambda x: x[0])
price_df["values"] = price_df["values"].astype(float)

In [0]:
price_df['time'] = pd.to_datetime(price_df['time'])
price_df['time'] = price_df['time'].dt.strftime('%d-%m-%Y')

In [0]:
price_df.head()

,time,values
0,18-07-2010,0.085840
1,19-07-2010,0.080800
2,20-07-2010,0.074736
3,21-07-2010,0.079193
4,22-07-2010,0.058470


## Metrics

In [0]:
start      = 1279324800
end        = 1524960000
metrics_df = pd.DataFrame()
api_key    = "5a0cf8d7-d14a-44f4-b063-a76807cd5340"
base       = "https://api.glassnode.com/v1"
url        = lambda api_key: f"{endpoint}?api_key={api_key}&a=BTC&s={start}&u={end}"
endpoints  = {
  "dormancy" : f"{base}/metrics/indicators/average_dormancy",
  "velocity" : f"{base}/metrics/indicators/velocity",
  "nvts"     : f"{base}/metrics/indicators/nvts",
  "sopr"     : f"{base}/metrics/indicators/sopr",
  "mvrv"     : f"{base}/metrics/market/mvrv" 
}

In [0]:
for name, endpoint in endpoints.items():
  response = make_request(url, api_key)
  tmp = pd.read_json(response.content, convert_dates=["t"], date_unit="s")
  tmp.columns = ["date", name]
  diff = tmp.columns.difference(metrics_df.columns)
  metrics_df = pd.concat([metrics_df, tmp[diff]], axis=1, sort=False)

https://api.glassnode.com/v1/metrics/indicators/average_dormancy?api_key=5a0cf8d7-d14a-44f4-b063-a76807cd5340&a=BTC&s=1279324800&u=1524960000
https://api.glassnode.com/v1/metrics/indicators/velocity?api_key=5a0cf8d7-d14a-44f4-b063-a76807cd5340&a=BTC&s=1279324800&u=1524960000
https://api.glassnode.com/v1/metrics/indicators/nvts?api_key=5a0cf8d7-d14a-44f4-b063-a76807cd5340&a=BTC&s=1279324800&u=1524960000
https://api.glassnode.com/v1/metrics/indicators/sopr?api_key=5a0cf8d7-d14a-44f4-b063-a76807cd5340&a=BTC&s=1279324800&u=1524960000
https://api.glassnode.com/v1/metrics/market/mvrv?api_key=5a0cf8d7-d14a-44f4-b063-a76807cd5340&a=BTC&s=1279324800&u=1524960000


### Join

In [0]:
metrics_df["target_price"] = price_df["values"]

In [0]:
metrics_df = metrics_df.set_index("date", drop=True)

In [0]:
metrics_df.head()

,dormancy,velocity,nvts,sopr,mvrv,target_price
date,,,,,,
2010-07-17,17.557391,0.009523,50.150020,1.000000,0.012190,0.085840
2010-07-18,37.898148,0.012957,65.319991,1.174760,0.015894,0.080800
2010-07-19,8.009993,0.016159,90.955940,1.318536,0.022353,0.074736
2010-07-20,2.325436,0.011241,79.371871,1.090517,0.019591,0.079193
2010-07-21,23.594425,0.010935,70.096419,1.065532,0.017317,0.058470


### Normalize

In [0]:
def normalize(df):
  norm = (df - df.mean()) / (df.max() - df.min())
  return norm

In [0]:
metrics_df = normalize(metrics_df) 

In [0]:
metrics_df.head()

,dormancy,velocity,nvts,sopr,mvrv,target_price
date,,,,,,
2010-07-17,0.053215,-0.027421,0.193640,-0.010630,-0.229075,-0.057378
2010-07-18,0.151102,-0.026649,0.282290,0.129321,-0.228566,-0.057378
2010-07-19,0.007270,-0.025929,0.432100,0.244460,-0.227678,-0.057378
2010-07-20,-0.020087,-0.027035,0.364405,0.061858,-0.228058,-0.057378
2010-07-21,0.082268,-0.027104,0.310202,0.041849,-0.228371,-0.057379


### Test-Training split

In [0]:
training_data, test_data = train_test_split(metrics_df, test_size=0.2, shuffle=False)
print(f"Training data size: {training_data.shape}, Testing data size: {test_data.shape}")

Training data size: (2274, 6), Testing data size: (569, 6)


In [0]:
training_data

,dormancy,velocity,nvts,sopr,mvrv,target_price
date,,,,,,
2010-07-17,0.053215,-0.027421,0.193640,-0.010630,-0.229075,-0.057378
2010-07-18,0.151102,-0.026649,0.282290,0.129321,-0.228566,-0.057378
2010-07-19,0.007270,-0.025929,0.432100,0.244460,-0.227678,-0.057378
2010-07-20,-0.020087,-0.027035,0.364405,0.061858,-0.228058,-0.057378
2010-07-21,0.082268,-0.027104,0.310202,0.041849,-0.228371,-0.057379
...,...,...,...,...,...,...
2016-10-02,-0.023238,-0.009494,-0.053498,-0.009859,-0.005268,-0.026192
2016-10-03,-0.012099,-0.001411,-0.053391,-0.006953,-0.005334,-0.026323
2016-10-04,0.000965,-0.002629,-0.053477,-0.007017,-0.006431,-0.026214


In [0]:
x_train = training_data[["dormancy", "velocity", "nvts", "sopr", "mvrv"]]
y_train = training_data["target_price"]

x_test = test_data[["dormancy", "velocity", "nvts", "sopr", "mvrv"]]
y_test = test_data["target_price"]

### Visualizing

### Model

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils import data
from torch.nn import functional as F
from torchvision import datasets, models, transforms

from sklearn.model_selection import train_test_split

In [0]:
EPOCHS = 1000
DROPOUT = 0.2
DIRECTIONS = 1
NUM_LAYERS = 2
BATCH_SIZE = 5
OUTPUT_SIZE = 1
SEQ_LENGTH = 90 # 90 day average
NUM_FEATURES = 5
HIDDEN_SIZE = 12
LEARNING_RATE = 0.0001

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

### Model definition

In [0]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
    super(LSTM, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
    # self.dropout = nn.Dropout(prob_dropout)
    self.linear = nn.Linear(hidden_size, output_size)

  def forward(self, x, states):
    x, (h, c) = self.lstm(x, states)
    out = self.linear(x)
    return out, (h, c)

In [0]:
model = LSTM(
    NUM_FEATURES,
    HIDDEN_SIZE,
    NUM_LAYERS,
    OUTPUT_SIZE,
    DROPOUT
).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.linear.parameters(), lr=LEARNING_RATE)

In [0]:
num_batches = len(x_train) // BATCH_SIZE // SEQ_LENGTH

In [0]:
def training(model, epochs, state_dim):
  
  for epoch in range(epochs):

      # Initialize states
      # (num_layers * num_directions, batch, hidden_size)
      states = (torch.zeros(state_dim).to(device), torch.zeros(state_dim).to(device))

      # Begin training
      for idx in range(num_batches):
        
          # Define range
          start = idx
          end = idx + SEQ_LENGTH * BATCH_SIZE
          x_window = x_train[start:end]
          y_window = y_train[start:end]

          # Convert to Tensors and modify dimensions
          x_tensor = torch.tensor(x_window.values).float()
          y_tensor = torch.tensor(y_window.values).float()
          x_batch = x_tensor.view(BATCH_SIZE, SEQ_LENGTH, NUM_FEATURES)
          y_batch = y_tensor.view(BATCH_SIZE, SEQ_LENGTH).unsqueeze(dim=2)
        
          # Move to GPU
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)

          optimizer.zero_grad()

          prediction, states = model(x_batch, states)

          # Detach states
          states = [state.detach() for state in states]
          
          loss = criterion(prediction, y_batch)
          loss.backward()
          optimizer.step()

          print('Epoch [{}/{}], Index: [{}], Loss: {:.4f}'
                .format(epoch+1, epochs, idx + 1, loss.item()))

In [0]:
training(model, EPOCHS, state_dim=(NUM_LAYERS * DIRECTIONS, BATCH_SIZE, HIDDEN_SIZE))

Epoch [1/1000], Index: [1], Loss: 0.0391
Epoch [1/1000], Index: [2], Loss: 0.0391
Epoch [1/1000], Index: [3], Loss: 0.0390
Epoch [1/1000], Index: [4], Loss: 0.0389
Epoch [1/1000], Index: [5], Loss: 0.0390
Epoch [2/1000], Index: [1], Loss: 0.0386
Epoch [2/1000], Index: [2], Loss: 0.0388
Epoch [2/1000], Index: [3], Loss: 0.0385
Epoch [2/1000], Index: [4], Loss: 0.0387
Epoch [2/1000], Index: [5], Loss: 0.0388
Epoch [3/1000], Index: [1], Loss: 0.0386
Epoch [3/1000], Index: [2], Loss: 0.0384
Epoch [3/1000], Index: [3], Loss: 0.0386
Epoch [3/1000], Index: [4], Loss: 0.0385
Epoch [3/1000], Index: [5], Loss: 0.0382
Epoch [4/1000], Index: [1], Loss: 0.0379
Epoch [4/1000], Index: [2], Loss: 0.0381
Epoch [4/1000], Index: [3], Loss: 0.0382
Epoch [4/1000], Index: [4], Loss: 0.0380
Epoch [4/1000], Index: [5], Loss: 0.0378
Epoch [5/1000], Index: [1], Loss: 0.0378
Epoch [5/1000], Index: [2], Loss: 0.0380
Epoch [5/1000], Index: [3], Loss: 0.0378
Epoch [5/1000], Index: [4], Loss: 0.0377
Epoch [5/1000], 

KeyboardInterrupt: ignored